In [28]:
from pathlib import Path
import utils
from typing import Callable
import scipy
from scipy import signal as sgn
import seaborn

In [2]:
def filtered_fft_features(signal_matrix):
    N = signal_matrix.shape[1]
    dt = 1/200
    T = N*dt
    sf = 200
    Q = 30
    f_notch = 50
    b_notch, a_notch = sgn.iirnotch(w0=f_notch, Q=Q, fs=sf)
    sig_notch = sgn.filtfilt(b_notch, a_notch, signal_matrix, axis=1)

    #Ahora creamos el filtro pasabanda Butterworth
    f_nq = sf/2
    f_low = 8
    f_high = 40
    order = 4
    b_band, a_band = sgn.iirfilter(
        N=order, Wn=[f_low/f_nq, f_high/f_nq], btype="bandpass", ftype="butter"
    )
    sig_filt = sgn.filtfilt(b_band, a_band, sig_notch, axis=1)

    fft = np.fft.rfft(sig_filt)
    Sxx = np.real(((2*dt**2)/T)*fft*fft.conj())
    return Sxx


def naif_fft_features(signal_matrix):
    N = signal_matrix.shape[1]
    dt = 1/200
    T = N*dt
    fft = np.fft.rfft(signal_matrix)
    Sxx = np.real(((2*dt**2)/T)*fft*fft.conj())
    return Sxx

def generate_df(data,features, chanels): 
    #le paso la data resultante de la extraccion de features, los nombres de los features en una lista y de 
    #los canales
    features_name=[]
    for ch in chanels:
        for feature in features:
            features_name.append(feature + 'ch ' + str (ch))
    return pd.DataFrame(data, columns=features_name)

In [3]:
class BCIDataset():
    def __init__(
        self, 
        csvs_path, 
        subject: str = 'all', 
        session: str = 'all',
        channel: str = 'all', 
        overlapping_fraction: float = 1/3, 
        window_size: int = 900,
        feature_extractor: Callable = naif_fft_features
    ):
        '''
        Object containing all examples from a time series from the dataset.
        Args:
            csvs_path (str): path al directorio donde estan los csv de los datos
            subject (str): sujeto a estudiar.
                Si se indica 'all' el dataset final generado tendrá ejemplos de todos los sujetos
            session (str): sesión a estudiar del sujeto seleccionado.
                si se indica 'all'  el dataset final generado tendrá ejemplos de todas las sesiones
            channel (str): 'ch0', 'ch1', 'ch2', 'ch3'. Si se indica 'all', los ejemplos serán la
                concatenación de los 4 canales.
            overlapping_fraction (float): porcentaje de desplazamiento de la "ventana" que hace el ejemplo
            window_size (int): tamaño de la ventana de tiempo que hace a un ejemplo (en muestras).
            feature_extractor (func): Función de extracción de features, le ingresa un arreglo
                (ejemplos en las filas, muestras en las columnas) y devuelve un arreglo (ejemplos en
                las filas y features en las columnas)
        '''
        self.csvs_path = Path(csvs_path)
        self.channel = channel
        self.parts = int(1 / overlapping_fraction)
        self.fraction = 1 / self.parts
        self.ws = window_size
        self.subject = subject
        self.session = session
        self.channels = ['ch0','ch1','ch2','ch3']
        self.feature_extractor = feature_extractor
        self.complete_dataset = utils.read_all_datasets(self.csvs_path)
        
        self.complete_examples_signal, self.complete_examples_features, \
        self.complete_labels, self.complete_metadata = \
            self.generate_examples()
        
    def generate_examples(self):
        # For each session generate the signal examples,
        # the feature extraction examples and labels arrays
        complete_examples_signal, complete_examples_features = [], []
        complete_labels, complete_metadata = [], []
        
        for subject in self.complete_dataset.subject.unique():
            # Just load the selected subject
            if self.subject != 'all' and subject != self.subject:
                continue
            sessions = \
                self.complete_dataset.loc[self.complete_dataset.subject==subject].session.unique()
            for session in sessions:
                # Just load the selected session
                if self.session != 'all' and session != self.session:
                    continue
                
                print(f'Processing subject: {subject} - session: {session}...')
                
                # Generate a subset of the dataset only with the desired rows
                selection = self.complete_dataset.loc[
                    (self.complete_dataset.subject==subject) &
                    (self.complete_dataset.session==session)
                ]
                
                # Standarize length of the array to a multiple to window size
                labels = selection.label.values
                n_rows = labels.shape[0] // self.ws
                labels = labels[: n_rows * self.ws]

                # Generate examples from the signal
                n_examples = n_rows * self.parts - (self.parts - 1)
                examples = np.empty((n_examples, self.ws))
                most_frec_labels = np.empty((n_examples, self.ws))
                times = np.empty((n_examples, self.ws))
                
                concat_ch_examples_signal, concat_ch_examples_features = [], []

                for k, ch in enumerate(self.channels):
                    # Use the four channels or just one
                    if self.channel != 'all' and self.channel != ch:
                        continue
                    
                    # Standarize length of the signal to a multiple to window size
                    signal = selection[ch].values
                    signal = signal[: n_rows * self.ws]
                    time = selection.time.values
                    time = time[: n_rows * self.ws]

                    # Increase the number of examples by overlapping the windows
                    for part in range(self.parts):
                        
                        # Find the place in the output array for each example
                        position = np.arange(part, n_examples, self.parts)
                        #position = position if part == 0 else position[:-part]

                        # Crop the signal according to the window size and overlap
                        start = int(self.ws / self.parts) * part
                        end = -int(self.ws - (self.ws / self.parts) * part)
                        end = end if part!=0 else signal.shape[0]
                        subset_signal = signal[start:end]
                        subset_labels = labels[start:end]
                        subset_times = time[start:end]

                        # Generate the examples
                        n_rows_ = int(subset_signal.shape[0]/self.ws)
                        examples[position, :] = subset_signal.reshape((n_rows_, self.ws))
                        most_frec_labels[position, :] = subset_labels.reshape((n_rows_, self.ws))
                        times[position, :] = subset_times.reshape((n_rows_, self.ws))
                    
                    # Obtain most frequent label
                    labels_ = scipy.stats.mode(most_frec_labels, axis=1).mode
                    labels_temp = scipy.stats.mode(most_frec_labels, axis=1).count
                    pureness = labels_temp == self.ws
                    # Get first and last time of the window
                    times_ = np.asarray([np.min(times, axis=1), np.max(times, axis=1)]).T
                    # Extract features
                    features = self.feature_extractor(examples)

                    concat_ch_examples_signal.append(examples.copy())
                    concat_ch_examples_features.append(features)

                concat_ch_examples_signal = np.concatenate(concat_ch_examples_signal, axis=1)
                concat_ch_examples_features = np.concatenate(concat_ch_examples_features, axis=1)

                complete_examples_signal.append(concat_ch_examples_signal)
                complete_examples_features.append(concat_ch_examples_features)
                complete_labels.append(labels_)
                lt = len(times_)
                metadata_ = np.concatenate(
                    [pureness, times_, np.repeat(subject, lt)[:,None], np.repeat(session, lt)[:,None]],
                    axis=1
                )
                complete_metadata.append(metadata_)

        complete_examples_signal = np.concatenate(complete_examples_signal)
        complete_examples_features = np.concatenate(complete_examples_features)
        complete_labels = np.concatenate(complete_labels)
        complete_metadata = np.concatenate(complete_metadata)
        
        return complete_examples_signal, complete_examples_features, complete_labels, complete_metadata
    
    def __len__(self):
        return self.complete_examples_signal.shape[0]
    
    def __getitem__(self, idx):
        return {
            'signal': self.complete_examples_signal[idx,:],
            'features': self.complete_examples_features[idx,:],
            'label': self.complete_labels[idx,:],
            'metadata': self.complete_metadata[idx,:]
        }

    def get_X_signal(self):
        return self.complete_examples_signal

    def get_X_features(self):
        return self.complete_examples_features

    def get_Y(self):
        return self.complete_labels

    def get_metadata(self):
        return self.complete_metadata

In [4]:
def calc_max_filtered(signal_matrix):
    N = signal_matrix.shape[1]
    dt = 1/200
    T = N*dt
    sf = 200
    Q = 30
    f_notch = 50
    b_notch, a_notch = sgn.iirnotch(w0=f_notch, Q=Q, fs=sf)
    sig_notch = sgn.filtfilt(b_notch, a_notch, signal_matrix, axis=1)

    #Ahora creamos el filtro pasabanda Butterworth
    f_nq = sf/2
    f_low = 9
    f_high = 22
    order = 4
    b_band, a_band = sgn.iirfilter(
        N=order, Wn=[f_low/f_nq, f_high/f_nq], btype="bandpass", ftype="butter"
    )
    sig_filt = sgn.filtfilt(b_band, a_band, sig_notch, axis=1)
    
    fft = np.fft.rfft(sig_filt)
    Sxx = np.real(((2*dt**2)/T)*fft*fft.conj())
    freqs = np.fft.rfftfreq(n=N, d=dt)
    
    maximum = np.argmax(Sxx, axis=1)
  
    item=freqs[maximum]
    return item.reshape((signal_matrix.shape[0], 1))

In [61]:
def filtered_range(signal_matrix):
    N = signal_matrix.shape[1]
    dt = 1/200
    T = N*dt
    sf = 200
    Q = 30
    f_notch = 50
    b_notch, a_notch = sgn.iirnotch(w0=f_notch, Q=Q, fs=sf)
    sig_notch = sgn.filtfilt(b_notch, a_notch, signal_matrix, axis=1)

    #Ahora creamos el filtro pasabanda Butterworth
    f_nq = sf/2
    f_low = 9
    f_high = 35
    order = 4
    b_band, a_band = sgn.iirfilter(
        N=order, Wn=[f_low/f_nq, f_high/f_nq], btype="bandpass", ftype="butter"
    )
    sig_filt = sgn.filtfilt(b_band, a_band, sig_notch, axis=1)

    fft = np.fft.rfft(sig_filt)
    Sxx = np.real(((2*dt**2)/T)*fft*fft.conj())
    freqs = np.fft.rfftfreq(n=N, d=dt)
    
    index_low=np.where(freqs>=10)[0][0]

    index_high=np.where(freqs>=35)[0][0]

    
    return Sxx[:,index_low:index_high]

In [6]:
def filtered_pca(signal_matrix):
    N = signal_matrix.shape[1]
    dt = 1/200
    T = N*dt
    sf = 200
    Q = 30
    f_notch = 50
    b_notch, a_notch = sgn.iirnotch(w0=f_notch, Q=Q, fs=sf)
    sig_notch = sgn.filtfilt(b_notch, a_notch, signal_matrix, axis=1)

    #Ahora creamos el filtro pasabanda Butterworth
    f_nq = sf/2
    f_low = 10
    f_high = 35
    order = 4
    b_band, a_band = sgn.iirfilter(
        N=order, Wn=[f_low/f_nq, f_high/f_nq], btype="bandpass", ftype="butter"
    )
    sig_filt = sgn.filtfilt(b_band, a_band, sig_notch, axis=1)

    fft = np.fft.rfft(sig_filt)
    Sxx = np.real(((2*dt**2)/T)*fft*fft.conj())
    freqs = np.fft.rfftfreq(n=N, d=dt)
    
    index_low=np.where(freqs>=10)[0][0]

    index_high=np.where(freqs>=35)[0][0]    
    
    from sklearn.decomposition import PCA
    pca = PCA(n_components=3)
    pca.fit(Sxx[:,index_low:index_high])
    proyected_features_freq = pca.transform(Sxx[:,index_low:index_high])
    
    return proyected_features_freq.reshape((signal_matrix.shape[0], 3))

In [7]:
csvs_path = '/home/ariel/gabriel/diplo_de_datos/mentoria_BCI/tp3'

In [62]:
# dataset con los valores de frecuencia donde aparece la amplitud máxima
dataset_filtered_max = BCIDataset(csvs_path,
                         window_size=750,
                        feature_extractor=calc_max_filtered)

Processing subject: MA - session: 1...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [63]:
# dataset con la senal filtrada y sólo teniendo el rango de frecuencias de interés
dataset_filtered_range = BCIDataset(csvs_path,
                         window_size=750,
                        feature_extractor=filtered_range)

Processing subject: MA - session: 1...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
# dataset con la senal filtrada y aplicando pca
dataset_filtered_pca = BCIDataset(csvs_path,
                         window_size=750,
                        feature_extractor=filtered_pca)

Processing subject: MA - session: 1...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [64]:
df_max_freq=generate_df(dataset_filtered_max.complete_examples_features,['max freq '], [0,1,2,3])

<IPython.core.display.Javascript object>

In [65]:

N=dataset_filtered_range.complete_examples_signal.shape[1]
dt = 1/200
freqs_all = np.fft.rfftfreq(n=int(N/4), d=dt)
index_low=np.where(freqs_all>=10)[0][0]
index_high=np.where(freqs_all>=35)[0][0]
name= [str(i) for i in list(freqs_all[index_low:index_high].round(2))]
df_range_freq=pd.DataFrame(dataset_filtered_range.complete_examples_features,columns=name+name+name+name)

df_range_freq['freq_label_1_mean_log']=np.log(df_range_freq[['12.0','12.27','12.53','12.8','13.07']].mean(axis=1))
df_range_freq['freq_label_2_mean_log']=np.log(df_range_freq[[ '16.0','16.27', '16.53', '16.8','17.07']].mean(axis=1))

df_range_freq['freq_label_1_median_log']=np.log(np.median(df_range_freq[['12.0','12.27','12.53','12.8','13.07']],axis=1))
df_range_freq['freq_label_2_median_log']=np.log(np.median(df_range_freq[[ '16.0','16.27', '16.53', '16.8','17.07']],axis=1))

df_range_freq['freq_label_1_sum_log'] = np.log(df_range_freq[['12.0','12.27','12.53','12.8','13.07']].sum(axis=1)/4)
df_range_freq['freq_label_2_sum_log'] = np.log(df_range_freq[[ '16.0','16.27', '16.53', '16.8','17.07']].sum(axis=1)/4)

df_range_freq['freq_label_1_quantile_log']=np.log(np.quantile(df_range_freq[['12.0','12.27','12.53','12.8','13.07']],0.9,axis=1))
df_range_freq['freq_label_2_quantile_log']=np.log(np.quantile(df_range_freq[[ '16.0','16.27', '16.53', '16.8','17.07']],0.9,axis=1))

df_range_freq['freq_label_1_std_log']=np.log(np.std(df_range_freq[['12.0','12.27','12.53','12.8','13.07']],axis=1))
df_range_freq['freq_label_2_std_log']=np.log(np.std(df_range_freq[[ '16.0','16.27', '16.53', '16.8','17.07']],axis=1))

df_range_freq['freq_label_1_var_log']=np.log(np.var(df_range_freq[['12.0','12.27','12.53','12.8','13.07']],axis=1))
df_range_freq['freq_label_2_var_log']=np.log(np.var(df_range_freq[[ '16.0','16.27', '16.53', '16.8','17.07']],axis=1))

df_range_freq['freq_label_1_entropy_log']=np.log(scipy.stats.entropy(df_range_freq[['12.0','12.27','12.53','12.8','13.07']],axis=1))
df_range_freq['freq_label_2_entropy_log']=np.log(scipy.stats.entropy(df_range_freq[[ '16.0','16.27', '16.53', '16.8','17.07']],axis=1))

df_range_freq['freq_label_1_skew']=scipy.stats.skew(df_range_freq[['12.0','12.27','12.53','12.8','13.07']],axis=1)
df_range_freq['freq_label_2_skew']=scipy.stats.skew(df_range_freq[[ '16.0','16.27', '16.53', '16.8','17.07']],axis=1)

df_range_freq['freq_label_1_kurtosis']=scipy.stats.kurtosis(df_range_freq[['12.0','12.27','12.53','12.8','13.07']],axis=1)
df_range_freq['freq_label_2_kurtosis']=scipy.stats.kurtosis(df_range_freq[[ '16.0','16.27', '16.53', '16.8','17.07']],axis=1)


#features sin log

df_range_freq['freq_label_1_mean']=df_range_freq[['12.0','12.27','12.53','12.8','13.07']].mean(axis=1)
df_range_freq['freq_label_2_mean']=df_range_freq[[ '16.0','16.27', '16.53', '16.8','17.07']].mean(axis=1)

df_range_freq['freq_label_1_median']=np.median(df_range_freq[['12.0','12.27','12.53','12.8','13.07']],axis=1)
df_range_freq['freq_label_2_median']=np.median(df_range_freq[[ '16.0','16.27', '16.53', '16.8','17.07']],axis=1)

df_range_freq['freq_label_1_sum'] = df_range_freq[['12.0','12.27','12.53','12.8','13.07']].sum(axis=1)/4
df_range_freq['freq_label_2_sum'] = df_range_freq[[ '16.0','16.27', '16.53', '16.8','17.07']].sum(axis=1)/4

df_range_freq['freq_label_1_quantile']=np.quantile(df_range_freq[['12.0','12.27','12.53','12.8','13.07']],0.9,axis=1)
df_range_freq['freq_label_2_quantile']=np.quantile(df_range_freq[[ '16.0','16.27', '16.53', '16.8','17.07']],0.9,axis=1)

df_range_freq['freq_label_1_std']=np.std(df_range_freq[['12.0','12.27','12.53','12.8','13.07']],axis=1)
df_range_freq['freq_label_2_std']=np.std(df_range_freq[[ '16.0','16.27', '16.53', '16.8','17.07']],axis=1)

df_range_freq['freq_label_1_var']=np.var(df_range_freq[['12.0','12.27','12.53','12.8','13.07']],axis=1)
df_range_freq['freq_label_2_var']=np.var(df_range_freq[[ '16.0','16.27', '16.53', '16.8','17.07']],axis=1)

df_range_freq['freq_label_1_entropy']=scipy.stats.entropy(df_range_freq[['12.0','12.27','12.53','12.8','13.07']],axis=1)
df_range_freq['freq_label_2_entropy']=scipy.stats.entropy(df_range_freq[[ '16.0','16.27', '16.53', '16.8','17.07']],axis=1)







array_freqs_label_1=df_range_freq[['12.0','12.27','12.53','12.8','13.07']].to_numpy()
array_freqs_label_2=df_range_freq[['16.0','16.27', '16.53', '16.8','17.07']].to_numpy()
#Hago una suma ponderada de los 4 canales en cjto. tener en cuenta que primero aparecen los valores 
#de cada frecuencia repetida por los cuatro canales
sum_weighted_1_log=np.log(np.dot(array_freqs_label_1,[0.1/4,0.1/4,0.1/4,0.1/4,
                                 0.2/4,0.2/4,0.2/4,0.2/4,
                                 0.4/4,0.4/4,0.4/4,0.4/4,
                                 0.2/4,0.2/4,0.2/4,0.2/4,
                                 0.1/4,0.1/4,0.1/4,0.1/4]))

sum_weighted_2_log=np.log(np.dot(array_freqs_label_2,[0.1/4,0.1/4,0.1/4,0.1/4,
                                 0.2/4,0.2/4,0.2/4,0.2/4,
                                 0.4/4,0.4/4,0.4/4,0.4/4,
                                 0.2/4,0.2/4,0.2/4,0.2/4,
                                 0.1/4,0.1/4,0.1/4,0.1/4]))

sum_weighted_1=np.dot(array_freqs_label_1,[0.1/4,0.1/4,0.1/4,0.1/4,
                                 0.2/4,0.2/4,0.2/4,0.2/4,
                                 0.4/4,0.4/4,0.4/4,0.4/4,
                                 0.2/4,0.2/4,0.2/4,0.2/4,
                                 0.1/4,0.1/4,0.1/4,0.1/4])

sum_weighted_2=np.dot(array_freqs_label_2,[0.1/4,0.1/4,0.1/4,0.1/4,
                                 0.2/4,0.2/4,0.2/4,0.2/4,
                                 0.4/4,0.4/4,0.4/4,0.4/4,
                                 0.2/4,0.2/4,0.2/4,0.2/4,
                                 0.1/4,0.1/4,0.1/4,0.1/4])

df_range_freq['sum_weighted_label_1']= sum_weighted_1.tolist()
df_range_freq['sum_weighted_label_2']= sum_weighted_2.tolist()
df_range_freq['sum_weighted_label_1_log']= sum_weighted_1_log.tolist()
df_range_freq['sum_weighted_label_2_log']= sum_weighted_2_log.tolist()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [66]:
N=dataset_filtered_range.complete_examples_signal.shape[1]
dt = 1/200
freqs_all = np.fft.rfftfreq(n=int(N/4), d=dt)
index_low=np.where(freqs_all>=10)[0][0]
index_high=np.where(freqs_all>=35)[0][0]
name= [str(i) for i in list(freqs_all[index_low:index_high].round(2))]
df_range_freq=pd.DataFrame(dataset_filtered_range.complete_examples_features,columns=name+name+name+name)

df_range_freq['freq_label_1_mean']=np.log(df_range_freq[['12.27','12.53','12.8']].mean(axis=1))
df_range_freq['freq_label_2_mean']=np.log(df_range_freq[[ '16.27', '16.53', '16.8']].mean(axis=1))

df_range_freq['freq_label_1_median']=np.log(np.median(df_range_freq[['12.27','12.53','12.8']],axis=1))
df_range_freq['freq_label_2_median']=np.log(np.median(df_range_freq[[ '16.27', '16.53', '16.8']],axis=1))

df_range_freq['freq_label_1_sum'] = np.log(df_range_freq[['12.27','12.53','12.8']].sum(axis=1)/4)
df_range_freq['freq_label_2_sum'] = np.log(df_range_freq[[ '16.27', '16.53', '16.8']].sum(axis=1)/4)

df_range_freq['freq_label_1_quantile']=np.log(np.quantile(df_range_freq[['12.27','12.53','12.8']],0.9,axis=1))
df_range_freq['freq_label_2_quantile']=np.log(np.quantile(df_range_freq[[ '16.27', '16.53', '16.8']],0.9,axis=1))

df_range_freq['freq_label_1_std']=np.log(np.std(df_range_freq[['12.27','12.53','12.8']],axis=1))
df_range_freq['freq_label_2_std']=np.log(np.std(df_range_freq[[ '16.27', '16.53', '16.8']],axis=1))

df_range_freq['freq_label_1_var']=np.log(np.var(df_range_freq[['12.27','12.53','12.8']],axis=1))
df_range_freq['freq_label_2_var']=np.log(np.var(df_range_freq[[ '16.27', '16.53', '16.8']],axis=1))


#df_range_freq['freq_label_1_quantile']=np.quantile(df_range_freq[['12.0','12.27','12.53','12.8','13.07']],0.9,axis=1)
#df_range_freq['freq_label_2_quantile']=np.quantile(df_range_freq[[ '16.0','16.27', '16.53', '16.8','17.07']],0.9,axis=1)
from tsfel.feature_extraction import features

df_range_freq['freq_label_1_entropy']=np.log(scipy.stats.entropy(df_range_freq[['12.27','12.53','12.8']],axis=1))
df_range_freq['freq_label_2_entropy']=np.log(scipy.stats.entropy(df_range_freq[[ '16.27', '16.53', '16.8']],axis=1))
df_range_freq['freq_label_1_skew']=scipy.stats.skew(df_range_freq[['12.27','12.53','12.8']],axis=1)
df_range_freq['freq_label_2_skew']=scipy.stats.skew(df_range_freq[[ '16.27', '16.53', '16.8']],axis=1)
df_range_freq['freq_label_1_kurtosis']=scipy.stats.kurtosis(df_range_freq[['12.27','12.53','12.8']],axis=1)
df_range_freq['freq_label_2_kurtosis']=scipy.stats.kurtosis(df_range_freq[[ '16.27', '16.53', '16.8']],axis=1)

array_freqs_label_1=df_range_freq[['12.27','12.53','12.8']].to_numpy()
array_freqs_label_2=df_range_freq[['16.27', '16.53', '16.8']].to_numpy()
#Hago una suma ponderada de los 4 canales en cjto. tener en cuenta que primero aparecen los valores 
#de cada frecuencia repetida por los cuatro canales
sum_weighted_1_log=np.log(np.dot(array_freqs_label_1,[
                                 0.2/4,0.2/4,0.2/4,0.2/4,
                                 0.6/4,0.6/4,0.6/4,0.6/4,
                                 0.2/4,0.2/4,0.2/4,0.2/4,
                                ]))

sum_weighted_2_log=np.log(np.dot(array_freqs_label_2,[
                                 0.2/4,0.2/4,0.2/4,0.2/4,
                                 0.6/4,0.6/4,0.6/4,0.6/4,
                                 0.2/4,0.2/4,0.2/4,0.2/4,
                                 ]))


df_range_freq['sum_weighted_label_1']= sum_weighted_1_log.tolist()
df_range_freq['sum_weighted_label_2']= sum_weighted_2_log.tolist()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
df_pca_freq=generate_df(dataset_filtered_pca.complete_examples_features,['pca 0 ', 'pca 1 ','pca 2 '], [0,1,2,3])


<IPython.core.display.Javascript object>

In [67]:
labels = pd.DataFrame(dataset_filtered_max.complete_labels,columns=['labels'])
metadata_all = pd.DataFrame(dataset_filtered_max.complete_metadata, columns = ['purness', 't_start', 't_end', 
                                                                      'subject', 'session'])

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [68]:
df_max_freq['max_freq_mean'] = df_max_freq.mean(axis=1)
df_max_freq['max_freq_mode'] = scipy.stats.mode(df_max_freq,axis=1)[0]
df_max_freq['max_freq_median']=np.median(df_max_freq,axis=1)
df_max_freq['max_freq_quantile']=np.quantile(df_max_freq,0.9,axis=1)
df_max_freq['max_freq_std']=np.std(df_max_freq,axis=1)
df_max_freq['max_freq_var']=np.var(df_max_freq,axis=1)
df_max_freq['max_freq_entropy'] = scipy.stats.entropy(df_max_freq,axis=1)
df_max_freq['max_freq_skew'] = scipy.stats.skew(df_max_freq,axis=1)

max_freq_with_labels = df_max_freq.merge(labels,left_index=True,right_index=True)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [69]:
freq_range_with_labels = df_range_freq.merge(labels,left_index=True,right_index=True)


In [69]:
pca_with_labels = df_pca_freq.merge(labels,left_index=True,right_index=True)


In [70]:
best_features=df_max_freq.merge(df_range_freq[['freq_label_1_mean','freq_label_2_mean','freq_label_1_sum','freq_label_2_sum',
          'freq_label_1_median','freq_label_2_median','freq_label_1_quantile','freq_label_2_quantile'
                                 ,'sum_weighted_label_1','sum_weighted_label_2','freq_label_1_std',
                                 'freq_label_2_std','freq_label_1_var','freq_label_2_var','freq_label_1_entropy',
                                 'freq_label_2_entropy','freq_label_1_skew','freq_label_2_skew','freq_label_1_kurtosis',
                                 'freq_label_2_kurtosis','12.27','12.53','12.8','16.27', '16.53', '16.8'
                                               ,'24.8', '25.07', '25.33','32.8','33.07', '33.33'
                                ]],left_index=True,right_index=True)
best_features["purness"]=metadata_all["purness"].replace({'True':1,'False':0})
best_features = best_features.merge(labels,left_index=True,right_index=True)
best_features


,max freq ch 0,max freq ch 1,max freq ch 2,max freq ch 3,max_freq_mean,max_freq_mode,max_freq_median,max_freq_quantile,max_freq_std,max_freq_var,...,33.07,33.07,33.07,33.07,33.33,33.33,33.33,33.33,purness,labels
0,13.066667,14.933333,14.666667,13.066667,13.933333,13.066667,13.500000,14.773333,7.630563e-01,17.499774,...,0.317562,0.272690,0.327152,0.387082,0.452623,0.010220,0.002635,0.021904,1,99.0
1,14.400000,14.400000,14.400000,14.400000,14.400000,14.400000,14.400000,14.400000,1.776357e-15,20.480000,...,3.144824,4.344609,3.509071,2.835151,4.912045,4.190681,4.418774,4.819769,1,99.0
2,15.733333,15.733333,15.733333,15.733333,15.733333,15.733333,15.733333,15.733333,0.000000e+00,24.448176,...,4.239237,4.181332,3.235163,3.174501,3.805201,2.497736,3.031566,3.721871,1,99.0
3,15.466667,15.466667,16.533333,16.266667,15.933333,15.466667,15.700000,16.373333,4.129223e-01,23.842956,...,4.863078,3.820568,2.272781,3.643924,3.892728,2.580542,4.275572,3.517393,1,99.0
4,10.133333,18.933333,10.133333,10.133333,12.333333,10.133333,10.133333,14.973333,3.048409e+00,16.376103,...,0.069657,0.338799,0.660315,0.267302,0.067059,0.096949,0.001505,0.016168,1,99.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2609,13.333333,11.200000,11.200000,13.600000,12.333333,11.200000,11.766667,13.440000,9.974715e-01,13.410402,...,0.087189,0.047166,0.001066,0.084995,0.686654,0.106599,0.079940,0.006577,1,1.0
2610,13.600000,19.733333,12.533333,13.866667,14.933333,12.533333,13.733333,16.853333,2.298074e+00,19.942456,...,0.594805,0.042757,0.082875,0.003399,0.277104,0.050448,0.015478,0.018676,1,1.0
2611,13.866667,16.800000,13.866667,13.866667,14.600000,13.866667,13.866667,15.480000,1.016136e+00,18.945899,...,0.473557,0.038235,0.007239,0.060590,0.118793,0.120577,0.146161,0.002751,1,1.0
2612,12.533333,15.733333,12.533333,18.666667,14.866667,12.533333,13.700000,16.906667,2.144199e+00,19.617688,...,0.321177,0.321703,0.158544,0.049334,0.017968,0.056742,0.012483,0.049059,1,1.0


### pruebo con features sin log

In [71]:
X, y = best_features.iloc[:, :-1], best_features.labels

In [72]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y) # se queda con el 10% para validación
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train) # del 90% hace 20% para train y 80% para test

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [55]:
y_train.value_counts()

99.0    1158
2.0      377
1.0      346
Name: labels, dtype: int64

In [32]:
X

,max freq ch 0,max freq ch 1,max freq ch 2,max freq ch 3,max_freq_mean,max_freq_mode,max_freq_median,max_freq_quantile,max_freq_std,max_freq_var,...,16.53,25.07,25.07,25.07,25.07,33.07,33.07,33.07,33.07,purness
0,13.066667,14.933333,14.666667,13.066667,13.933333,13.066667,13.500000,14.773333,7.630563e-01,17.499774,...,1.069708,0.005906,0.037174,0.027149,0.012772,0.000891,0.003413,0.001621,0.001370,1
1,14.400000,14.400000,14.400000,14.400000,14.400000,14.400000,14.400000,14.400000,1.776357e-15,20.480000,...,0.586459,0.061410,0.045270,0.035993,0.036742,0.001662,0.003505,0.001339,0.001531,1
2,15.733333,15.733333,15.733333,15.733333,15.733333,15.733333,15.733333,15.733333,0.000000e+00,24.448176,...,2.965765,0.054243,0.036026,0.022648,0.027344,0.000231,0.000143,0.000198,0.000157,1
3,15.466667,15.466667,16.533333,16.266667,15.933333,15.466667,15.700000,16.373333,4.129223e-01,23.842956,...,2.094231,0.045852,0.079123,0.024660,0.031169,0.000031,0.000362,0.000045,0.000094,1
4,10.133333,18.933333,10.133333,10.133333,12.333333,10.133333,10.133333,14.973333,3.048409e+00,16.376103,...,0.266700,0.002842,0.001923,0.000047,0.000172,0.000047,0.000020,0.000016,0.000006,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2609,13.333333,11.200000,11.200000,13.600000,12.333333,11.200000,11.766667,13.440000,9.974715e-01,13.410402,...,1.974198,0.007321,0.008107,0.040299,0.018752,0.000132,0.000151,0.000092,0.000052,1
2610,13.600000,19.733333,12.533333,13.866667,14.933333,12.533333,13.733333,16.853333,2.298074e+00,19.942456,...,3.366331,0.004616,0.019391,0.038491,0.014786,0.000439,0.000583,0.000612,0.000386,1
2611,13.866667,16.800000,13.866667,13.866667,14.600000,13.866667,13.866667,15.480000,1.016136e+00,18.945899,...,2.219814,0.004271,0.014865,0.032814,0.019479,0.001724,0.002399,0.001407,0.003239,1
2612,12.533333,15.733333,12.533333,18.666667,14.866667,12.533333,13.700000,16.906667,2.144199e+00,19.617688,...,0.870207,0.004458,0.013135,0.055464,0.021754,0.000136,0.000480,0.000153,0.000293,1


In [33]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report
target_names = ['1', '2','99']


dc = DummyClassifier(strategy='uniform')
dc.fit(X_train, y_train)
y_train_pred=dc.predict(X_train)
print(classification_report(y_train, y_train_pred ))

              precision    recall  f1-score   support

         1.0       0.20      0.37      0.26       346
         2.0       0.21      0.38      0.27       377
        99.0       0.61      0.31      0.41      1158

    accuracy                           0.33      1881
   macro avg       0.34      0.35      0.31      1881
weighted avg       0.45      0.33      0.35      1881



In [34]:
y_val_pred = dc.predict(X_val)
print(classification_report(y_val, y_val_pred))

              precision    recall  f1-score   support

         1.0       0.15      0.29      0.20        48
         2.0       0.26      0.42      0.32        53
        99.0       0.65      0.34      0.44       161

    accuracy                           0.34       262
   macro avg       0.35      0.35      0.32       262
weighted avg       0.48      0.34      0.37       262



In [35]:
y_test_pred = dc.predict(X_test)
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

         1.0       0.19      0.31      0.24        87
         2.0       0.19      0.35      0.24        94
        99.0       0.62      0.33      0.43       290

    accuracy                           0.33       471
   macro avg       0.33      0.33      0.30       471
weighted avg       0.45      0.33      0.36       471



In [46]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

OVR=OneVsRestClassifier(LinearSVC(random_state=0))
OVR.fit(X_train, y_train)
y_train_pred = OVR.predict(X_train)
y_val_pred = OVR.predict(X_val)

/home/ariel/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:1199: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ariel/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:1199: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ariel/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:1199: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [47]:
from sklearn.metrics import classification_report
print(classification_report(y_train, y_train_pred))


              precision    recall  f1-score   support

         1.0       0.49      0.65      0.56       346
         2.0       0.95      0.05      0.09       377
        99.0       0.70      0.85      0.77      1158

    accuracy                           0.65      1881
   macro avg       0.71      0.52      0.47      1881
weighted avg       0.71      0.65      0.59      1881



In [48]:
print(classification_report(y_val, y_val_pred))

              precision    recall  f1-score   support

         1.0       0.39      0.56      0.46        48
         2.0       0.67      0.04      0.07        53
        99.0       0.68      0.80      0.74       161

    accuracy                           0.60       262
   macro avg       0.58      0.47      0.42       262
weighted avg       0.62      0.60      0.55       262



In [29]:
y_test_pred = OVR.predict(X_test)
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

         1.0       1.00      0.07      0.13        87
         2.0       0.56      0.46      0.50        94
        99.0       0.68      0.91      0.78       290

    accuracy                           0.66       471
   macro avg       0.75      0.48      0.47       471
weighted avg       0.71      0.66      0.60       471



from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
cm=confusion_matrix(y_test,y_test_pred)
print('matriz de confusion test')
print(cm)
#tn, fp, fn, tp = confusion_matrix(y_test,y_test_pred).ravel()
print('matriz de confusion test')
plot_confusion_matrix(OVR.fit(X_train, y_train), X_test, y_test,
                                 display_labels=target_names,
                                 cmap=plt.cm.Blues,
                                 normalize='true')

In [49]:
from sklearn.multiclass import OneVsOneClassifier
OVO=OneVsOneClassifier(LinearSVC(random_state=0))
OVO.fit(X_train, y_train)
y_train_pred = OVO.predict(X_train)
y_val_pred = OVO.predict(X_val)

/home/ariel/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:1199: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ariel/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:1199: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ariel/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:1199: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [50]:
print(classification_report(y_val, y_val_pred))

              precision    recall  f1-score   support

         1.0       0.41      0.77      0.53        48
         2.0       0.55      0.21      0.30        53
        99.0       0.70      0.66      0.68       161

    accuracy                           0.59       262
   macro avg       0.55      0.55      0.50       262
weighted avg       0.62      0.59      0.58       262



In [51]:
y_test_pred = OVO.predict(X_test)
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

         1.0       0.49      0.84      0.62        87
         2.0       0.73      0.23      0.35        94
        99.0       0.73      0.74      0.74       290

    accuracy                           0.66       471
   macro avg       0.65      0.60      0.57       471
weighted avg       0.69      0.66      0.64       471



### Random forest

In [73]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
param = {
    'n_estimators':[100,150,200],
    'criterion':['gini','entropy'],
    #'max_depth':[25,20,None,10],
    'min_samples_split':[2,5,None]
}
param_grid = {
  'n_estimators': [150],
  'max_features': [5, 7, 9],
  'max_depth'   : [None, 3, 10, 20],
  'criterion'   : ['gini', 'entropy']
}
#cv = GridSearchCV(estimator=rfc,param_grid=param, scoring='f1_micro',n_jobs=-1,cv=5, verbose=1)
rfc.fit(X_train, y_train)
y_train_pred = rfc.predict(X_train)
y_val_pred = rfc.predict(X_val)
y_test_pred = rfc.predict(X_test)
print('reporte train')
print(classification_report(y_train, y_train_pred))
print('reporte validacion')
print(classification_report(y_val, y_val_pred))
print('reporte test')
print(classification_report(y_test, y_test_pred))

reporte train
              precision    recall  f1-score   support

         1.0       1.00      1.00      1.00       346
         2.0       1.00      1.00      1.00       377
        99.0       1.00      1.00      1.00      1158

    accuracy                           1.00      1881
   macro avg       1.00      1.00      1.00      1881
weighted avg       1.00      1.00      1.00      1881

reporte validacion
              precision    recall  f1-score   support

         1.0       0.84      0.65      0.73        48
         2.0       0.74      0.43      0.55        53
        99.0       0.76      0.91      0.83       161

    accuracy                           0.77       262
   macro avg       0.78      0.66      0.70       262
weighted avg       0.77      0.77      0.75       262

reporte test
              precision    recall  f1-score   support

         1.0       0.82      0.70      0.76        87
         2.0       0.84      0.40      0.55        94
        99.0       0.77      

In [74]:
importances = pd.DataFrame({'feature':X_train.columns,'importance':np.round(rfc.feature_importances_,3)*100})
importances = importances.sort_values('importance',ascending=False)
importances

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,feature,importance
62,25.07,6.6
63,25.07,3.7
78,33.33,3.1
38,12.53,3.1
69,32.8,2.5
...,...,...
5,max_freq_mode,0.6
29,freq_label_2_skew,0.6
1,max freq ch 1,0.6
6,max_freq_median,0.5


In [34]:
best_tree_clf = cv.best_estimator_
best_tree_clf

NameError: name 'cv' is not defined

In [ ]:
cv.best_params_

### Support vector machine

In [78]:
from sklearn.svm import SVC

svc_model = SVC(gamma='auto',random_state=100)
svc_model.fit(X_train, y_train)

y_train_pred = svc_model.predict(X_train)
y_val_pred = svc_model.predict(X_val)
y_test_pred = svc_model.predict(X_test)

print('reporte train')
print(classification_report(y_train, y_train_pred))
print('reporte validacion')
print(classification_report(y_val, y_val_pred))
print('reporte test')
print(classification_report(y_test, y_test_pred))

reporte train
              precision    recall  f1-score   support

         1.0       0.94      0.84      0.89       346
         2.0       0.94      0.59      0.73       377
        99.0       0.84      0.97      0.90      1158

    accuracy                           0.87      1881
   macro avg       0.91      0.80      0.84      1881
weighted avg       0.88      0.87      0.87      1881

reporte validacion
              precision    recall  f1-score   support

         1.0       0.82      0.69      0.75        48
         2.0       0.70      0.30      0.42        53
        99.0       0.74      0.92      0.82       161

    accuracy                           0.75       262
   macro avg       0.75      0.64      0.66       262
weighted avg       0.75      0.75      0.73       262

reporte test
              precision    recall  f1-score   support

         1.0       0.81      0.64      0.72        87
         2.0       0.82      0.29      0.43        94
        99.0       0.73      

### gradient boosting

In [75]:

from sklearn.ensemble import GradientBoostingClassifier

# GradientBoostingClassifier
gbc = GradientBoostingClassifier()
gbc.fit(X_train, y_train)
y_train_pred = gbc.predict(X_train)
y_val_pred = gbc.predict(X_val)
y_test_pred = gbc.predict(X_test)
print('reporte train')
print(classification_report(y_train, y_train_pred))
print('reporte validacion')
print(classification_report(y_val, y_val_pred))
print('reporte test')
print(classification_report(y_test, y_test_pred))

reporte train
              precision    recall  f1-score   support

         1.0       0.96      0.93      0.95       346
         2.0       0.98      0.84      0.90       377
        99.0       0.93      0.98      0.96      1158

    accuracy                           0.94      1881
   macro avg       0.96      0.92      0.94      1881
weighted avg       0.95      0.94      0.94      1881

reporte validacion
              precision    recall  f1-score   support

         1.0       0.82      0.77      0.80        48
         2.0       0.74      0.64      0.69        53
        99.0       0.82      0.88      0.85       161

    accuracy                           0.81       262
   macro avg       0.80      0.76      0.78       262
weighted avg       0.81      0.81      0.81       262

reporte test
              precision    recall  f1-score   support

         1.0       0.81      0.76      0.79        87
         2.0       0.75      0.51      0.61        94
        99.0       0.80      

In [80]:
importances = pd.DataFrame({'feature':X_train.columns,'importance':np.round(gbc.feature_importances_,3)*100})
importances = importances.sort_values('importance',ascending=False)
importances.head(50)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,feature,importance
62,25.07,19.5
69,32.8,5.8
78,33.33,4.9
2,max freq ch 2,4.3
73,33.07,4.1
74,33.07,3.5
79,33.33,2.8
68,32.8,2.8
54,16.8,2.6
63,25.07,2.5


In [87]:
features=importances.head(15).feature.tolist()

In [53]:
from sklearn.pipeline import make_pipeline

gbc = make_pipeline(StandardScaler(), GradientBoostingClassifier(random_state=100))
# GradientBoostingClassifier
#gbc = GradientBoostingClassifier(random_state=10)
gbc.fit(X_train, y_train)
y_train_pred = gbc.predict(X_train)
y_val_pred = gbc.predict(X_val)
y_test_pred = gbc.predict(X_test)
print('reporte train')
print(classification_report(y_train, y_train_pred))
print('reporte validacion')
print(classification_report(y_val, y_val_pred))
print('reporte test')
print(classification_report(y_test, y_test_pred))

<IPython.core.display.Javascript object>

reporte train
              precision    recall  f1-score   support

         1.0       0.87      0.66      0.75       346
         2.0       0.91      0.58      0.71       377
        99.0       0.81      0.97      0.88      1158

    accuracy                           0.83      1881
   macro avg       0.86      0.73      0.78      1881
weighted avg       0.84      0.83      0.82      1881

reporte validacion
              precision    recall  f1-score   support

         1.0       0.71      0.42      0.53        48
         2.0       0.61      0.38      0.47        53
        99.0       0.70      0.88      0.78       161

    accuracy                           0.69       262
   macro avg       0.67      0.56      0.59       262
weighted avg       0.68      0.69      0.67       262

reporte test
              precision    recall  f1-score   support

         1.0       0.64      0.47      0.54        87
         2.0       0.64      0.39      0.49        94
        99.0       0.72      

### SGDC

In [42]:
from sklearn.linear_model import SGDClassifier, LogisticRegression
sgdc = SGDClassifier()
sgdc.fit(X_train, y_train)
y_train_pred = sgdc.predict(X_train)
y_val_pred = sgdc.predict(X_val)
y_test_pred = sgdc.predict(X_test)
print('reporte train')
print(classification_report(y_train, y_train_pred))
print('reporte validacion')
print(classification_report(y_val, y_val_pred))
print('reporte test')
print(classification_report(y_test, y_test_pred))

reporte train
              precision    recall  f1-score   support

         1.0       0.13      0.12      0.13       346
         2.0       0.23      0.23      0.23       377
        99.0       0.58      0.59      0.59      1158

    accuracy                           0.43      1881
   macro avg       0.31      0.31      0.31      1881
weighted avg       0.43      0.43      0.43      1881

reporte validacion
              precision    recall  f1-score   support

         1.0       0.08      0.06      0.07        48
         2.0       0.30      0.30      0.30        53
        99.0       0.56      0.58      0.57       161

    accuracy                           0.43       262
   macro avg       0.31      0.32      0.31       262
weighted avg       0.42      0.43      0.42       262

reporte test
              precision    recall  f1-score   support

         1.0       0.09      0.10      0.10        87
         2.0       0.29      0.22      0.25        94
        99.0       0.60      

### logistic regresion

In [79]:
from sklearn.linear_model import LogisticRegression
lrc = LogisticRegression(random_state=100)
lrc.fit(X_train, y_train)
y_train_pred = lrc.predict(X_train)
y_val_pred = lrc.predict(X_val)
y_test_pred = lrc.predict(X_test)
print('reporte train')
print(classification_report(y_train, y_train_pred))
print('reporte validacion')
print(classification_report(y_val, y_val_pred))
print('reporte test')
print(classification_report(y_test, y_test_pred))

reporte train
              precision    recall  f1-score   support

         1.0       0.83      0.67      0.74       346
         2.0       0.72      0.45      0.55       377
        99.0       0.77      0.90      0.83      1158

    accuracy                           0.77      1881
   macro avg       0.77      0.68      0.71      1881
weighted avg       0.77      0.77      0.76      1881

reporte validacion
              precision    recall  f1-score   support

         1.0       0.76      0.77      0.76        48
         2.0       0.71      0.57      0.63        53
        99.0       0.81      0.86      0.83       161

    accuracy                           0.78       262
   macro avg       0.76      0.73      0.74       262
weighted avg       0.78      0.78      0.78       262

reporte test
              precision    recall  f1-score   support

         1.0       0.82      0.72      0.77        87
         2.0       0.72      0.49      0.58        94
        99.0       0.78      

/home/ariel/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### decision tree

In [44]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(random_state=100)
dtc.fit(X_train, y_train)
y_train_pred = dtc.predict(X_train)
y_val_pred = dtc.predict(X_val)
y_test_pred = dtc.predict(X_test)
print('reporte train')
print(classification_report(y_train, y_train_pred))
print('reporte validacion')
print(classification_report(y_val, y_val_pred))
print('reporte test')
print(classification_report(y_test, y_test_pred))

reporte train
              precision    recall  f1-score   support

         1.0       1.00      1.00      1.00       346
         2.0       1.00      1.00      1.00       377
        99.0       1.00      1.00      1.00      1158

    accuracy                           1.00      1881
   macro avg       1.00      1.00      1.00      1881
weighted avg       1.00      1.00      1.00      1881

reporte validacion
              precision    recall  f1-score   support

         1.0       0.37      0.40      0.38        48
         2.0       0.32      0.32      0.32        53
        99.0       0.68      0.67      0.68       161

    accuracy                           0.55       262
   macro avg       0.46      0.46      0.46       262
weighted avg       0.55      0.55      0.55       262

reporte test
              precision    recall  f1-score   support

         1.0       0.53      0.57      0.55        87
         2.0       0.44      0.51      0.48        94
        99.0       0.73      

### balanceo de datos


In [47]:
#pruebo balanceando el dataset sacando muestras 99
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
over_sampler = SMOTE(k_neighbors=2)
X_res, y_res = over_sampler.fit_resample(X_train, y_train)
#print(f"Training target statistics: {Counter(y_res)}")
#print(f"Testing target statistics: {Counter(y_test)}")

In [48]:
y_res.value_counts()

99.0    1158
2.0     1158
1.0     1158
Name: labels, dtype: int64

In [52]:
from sklearn.ensemble import GradientBoostingClassifier
# GradientBoostingClassifier
gbc = GradientBoostingClassifier(random_state=10)
gbc.fit(X_res, y_res)
y_train_pred = gbc.predict(X_res)
y_val_pred = gbc.predict(X_val)
y_test_pred = gbc.predict(X_test)
print('reporte train')
print(classification_report(y_res, y_train_pred))
print('reporte validacion')
print(classification_report(y_val, y_val_pred))
print('reporte test')
print(classification_report(y_test, y_test_pred))

reporte train
              precision    recall  f1-score   support

         1.0       0.86      0.91      0.88      1158
         2.0       0.85      0.88      0.86      1158
        99.0       0.88      0.79      0.83      1158

    accuracy                           0.86      3474
   macro avg       0.86      0.86      0.86      3474
weighted avg       0.86      0.86      0.86      3474

reporte validacion
              precision    recall  f1-score   support

         1.0       0.45      0.60      0.52        48
         2.0       0.53      0.58      0.55        53
        99.0       0.74      0.64      0.69       161

    accuracy                           0.62       262
   macro avg       0.57      0.61      0.59       262
weighted avg       0.64      0.62      0.63       262

reporte test
              precision    recall  f1-score   support

         1.0       0.50      0.63      0.56        87
         2.0       0.54      0.57      0.56        94
        99.0       0.78      